# 필요 함수 import

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

# 1차 : 쿠팡 상품 리스트 추출
> categoryId
- 밀키트 : 502482<br>
//
- 1국탕전골 : 502483
- 2덮밥/비빔밥 : 502484<br>
//
- 3스테이크/고기 502485
- 4면/파스타/감바스 502486
- 5분식 502487
- 6중식요리 502490
<br>//
- 7기타요리 502491

In [2]:
# 쿠팡 전체 html 추출 함수
def get_coupang_item(URL, user_agt):
    headers = {"User-Agent":user_agt
               , "Accept-Language": "ko-KR,ko;q=0.8,en-US;q=0.5,en;q=0.3"}
    res = requests.get(URL, headers=headers)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, 'lxml')
    return soup

## 필요 정보만 추출
- 카테고리 대분류, 상품id, 상품only, 상품명(전체), 정가, 할인율, 판매가, 100g당_가격, 별점, 리뷰수, 품절여부

In [3]:
#카테고리 대분류 라벨링 함수
def category_number(categoryId, category):
    return category.get(categoryId)


#1차 쿠팡 상품 리스트 전처리 함수
def first_preprocessing(cg_soup, category_num):
    #상품id, data-item-id, data-vendor-item-id
    bady_pdt = cg_soup.select_one("a.baby-product-link")
    productId = bady_pdt['data-product-id']
    itemsId = bady_pdt['data-item-id']
    vendorItemId = bady_pdt['data-vendor-item-id']

    # 상품명
    name = cg_soup.select_one("div.name").text.replace('\n', '').strip()
    name_only = name.split(', ')[0]

    # 판매가
    price_value = cg_soup.select_one("strong.price-value").text.replace(',','')
    
    try:
        # 정가
        base_price = cg_soup.select_one("del.base-price").text.replace("\n",'').strip().replace(',','')
        # 할인율
        discount_pcg = cg_soup.select_one("span.discount-percentage").text
    except:
        base_price, discount_pcg = price_value, 0

    #100g당 가격
    try:
        unit_price = cg_soup.select_one("span.unit-price").text.replace("\n", '').strip()
    except:
        unit_price = np.NaN
    
    try:
        # 별점
        star = cg_soup.select_one("span.star > em").text
        # 리뷰수
        rating_count = cg_soup.select_one("span.rating-total-count").text
        rating_count = re.sub(r'[^0-9]','',rating_count)
    except:
        star, rating_count = 0, 0

    # 품절여부 > 품절이면 1, 아니면 0
    try:
        out_of_stock = cg_soup.select_one("div.out-of-stock").text.replace("\n", '').strip()
        out_of_stock = 1
    except:
        out_of_stock = 0
    
    #DataFrame으로 병합
    coupang_item = pd.DataFrame({'카테고리명':category_num,'상품id':productId, 'data-item-id':itemsId, 'data-vendor-item-id':vendorItemId
                                , '상품':name_only,'상품명':name, '정가':base_price, '할인율':discount_pcg, '판매가':price_value
                                , '100g당_가격':unit_price, '별점':star, '리뷰수':rating_count, '품절여부':out_of_stock}
                                , index=[0])
    return coupang_item


#해당 카테고리의 페이지 전체 수 가져오기
def page_number(categoryId, user_agt):
    URL = 'https://www.coupang.com/np/categories/'+categoryId+"?page=1"
    cg_soup_base = get_coupang_item(URL, user_agt)
    page_all = cg_soup_base.select('#product-list-paging > div > a')
    page_list = []
    for page_num in page_all:
        page_num = page_num.text
        try:
            page_num = int(page_num)
            page_list.append(page_num)
        except:
            pass
    return page_list[-1]

## 쿠팡 상품 리스트 추출 실행부

In [4]:
#필터링 후 전체 크롤링
#categoryId 기반 카테고리 대분류 정의
category = {'502483':1, '502484':2, '502485':3, '502486':4, '502487':5, '502490':6, '502491':7}
category_keys = list(category.keys())

#상품 리스트를 담을 빈 dataframe 생성
item_list = ['카테고리명','상품id','data-item-id','data-vendor-item-id','상품','상품명','정가',
             '할인율','판매가','100g당_가격','별점','리뷰수','품절여부']
coupang_items = pd.DataFrame(columns=item_list)

#쿠팡 상품 리스트 페이지 html 추출
cg_url = 'https://www.coupang.com/np/categories/'
user_agt = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.46"

for categoryId in category_keys:
    page_num = page_number(categoryId, user_agt) #해당 카테고리의 페이지 마지막 페이지 번호 추출
    for page in range(1, page_num+1):
        URL = cg_url+categoryId+"?page="+str(page)+'&filter=1%23attr_12406%2419087%40DEFAULT'
        cg_soup_base = get_coupang_item(URL, user_agt)     #전체 html 추출
        cg_soup2 = cg_soup_base.select("#productList li")  

        #코드 실행
        for cg_soup in cg_soup2:
            coupang_item = first_preprocessing(cg_soup, category_number(categoryId, category))
            coupang_items = pd.concat([coupang_items,coupang_item], ignore_index=True)

coupang_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 369 entries, 0 to 368
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   카테고리명                369 non-null    object
 1   상품id                 369 non-null    object
 2   data-item-id         369 non-null    object
 3   data-vendor-item-id  369 non-null    object
 4   상품                   369 non-null    object
 5   상품명                  369 non-null    object
 6   정가                   369 non-null    object
 7   할인율                  369 non-null    object
 8   판매가                  369 non-null    object
 9   100g당_가격             328 non-null    object
 10  별점                   369 non-null    object
 11  리뷰수                  369 non-null    object
 12  품절여부                 369 non-null    object
dtypes: object(13)
memory usage: 37.6+ KB


In [15]:
#카테고리별 중복값 확인
coupang_items.loc[coupang_items.duplicated(subset=['상품id']), :]

,카테고리명,상품id,data-item-id,data-vendor-item-id,상품,상품명,정가,할인율,판매가,100g당_가격,별점,리뷰수,품절여부


# 2차 : 구성 정보 추출
- 메뉴이름, 구성정보

In [6]:
data_items = pd.DataFrame(columns=['상품id', '구성정보'])

for idx in range(len(coupang_items)):
    product_url = 'https://www.coupang.com/vp/products/'
    productId = coupang_items.loc[idx, '상품id'] #data-product-id
    itemsId = coupang_items.loc[idx, 'data-item-id']  #data-item-id
    vendorItemId = coupang_items.loc[idx, 'data-vendor-item-id'] #data-vendor-item-id
    URL = product_url+str(productId)+'/items/'+str(itemsId)+'/vendoritems/'+str(vendorItemId)
    
    pdt_soup = get_coupang_item(URL, user_agt)
    
    all = pdt_soup.find('body').children
    info_of_organization = 0
    for item in all:
        s = item.text.strip().replace('\n','')
        if '로 구성' in s:
            info_of_organization = s
        elif '구성과 단위' in s : 
            info_of_organization=s
            
    #4차: 구성정보가 없으면 결측값 처리
    if info_of_organization==0:
        info_of_organization = np.NaN
    
    data_item = pd.DataFrame({'상품id':productId, '구성정보':info_of_organization}, index=[0])
    data_items = pd.concat([data_items, data_item], ignore_index=True)

data_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 369 entries, 0 to 368
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   상품id    369 non-null    object
 1   구성정보    330 non-null    object
dtypes: object(2)
memory usage: 5.9+ KB


In [7]:
coupang = pd.merge(coupang_items, data_items)
coupang.to_csv('coupang_filtering.csv')

In [8]:
coupang.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 369 entries, 0 to 368
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   카테고리명                369 non-null    object
 1   상품id                 369 non-null    object
 2   data-item-id         369 non-null    object
 3   data-vendor-item-id  369 non-null    object
 4   상품                   369 non-null    object
 5   상품명                  369 non-null    object
 6   정가                   369 non-null    object
 7   할인율                  369 non-null    object
 8   판매가                  369 non-null    object
 9   100g당_가격             328 non-null    object
 10  별점                   369 non-null    object
 11  리뷰수                  369 non-null    object
 12  품절여부                 369 non-null    object
 13  구성정보                 330 non-null    object
dtypes: object(14)
memory usage: 43.2+ KB


In [9]:
coupang.loc[coupang['구성정보'].isnull()]

,카테고리명,상품id,data-item-id,data-vendor-item-id,상품,상품명,정가,할인율,판매가,100g당_가격,별점,리뷰수,품절여부,구성정보
6,1,1637283332,2792368476,70782064297,곰곰 우삼겹 순두부 찌개,"곰곰 우삼겹 순두부 찌개, 710g, 1세트",8190,2%,7990,"(100g당 1,125원)",4.5,14162,0,NaN
7,1,4926044090,6464437521,73758857275,곰곰 밀푀유 나베,"곰곰 밀푀유 나베, 1.2kg, 1세트",14990,0,14990,"(100g당 1,249원)",4.5,15515,0,NaN
8,1,1946869683,3305331651,71292240570,곰곰 애호박 된장찌개 밀키트,"곰곰 애호박 된장찌개 밀키트, 730g, 1개",8170,2%,7990,"(100g당 1,095원)",4.5,6441,0,NaN
11,1,2251401014,3848979135,71833775592,곰곰 간편한 청국장찌개 밀키트,"곰곰 간편한 청국장찌개 밀키트, 770g, 1개",11180,0,11180,"(100g당 1,452원)",4.5,4510,0,NaN
16,1,5026557256,6745912116,74038795689,곰곰 소고기 된장찌개 밀키트,"곰곰 소고기 된장찌개 밀키트, 610g, 1개",7900,14%,6780,"(100g당 1,111원)",5.0,4649,0,NaN
19,1,2322032858,4010406015,71994624450,곰곰 옛날식 부대찌개,"곰곰 옛날식 부대찌개, 878g, 1개",14900,0,14900,"(100g당 1,697원)",4.5,5768,0,NaN
22,1,4923517179,6455403350,73750186590,곰곰 깊은맛 곱창전골 밀키트,"곰곰 깊은맛 곱창전골 밀키트, 1.596kg, 1개",22990,0,22990,"(100g당 1,440원)",4.5,5902,0,NaN
24,1,1421897672,2460513791,70454023419,프레시밀 감자 짜글이,"프레시밀 감자 짜글이, 770g, 1팩",12500,0,12500,"(100g당 1,623원)",4.5,1378,0,NaN
62,1,1717572551,2923204602,70911802212,프레시지 더큰 우삼겹 순두부찌개,"프레시지 더큰 우삼겹 순두부찌개, 1215g, 1개",13900,0,13900,"(100g당 1,144원)",4.5,7368,0,NaN
113,1,4841176792,6256834516,73552540510,곰곰 샤브 버섯 국수 전골 (냉장),"곰곰 샤브 버섯 국수 전골 (냉장), 1.01kg, 1개",12900,10%,11490,"(100g당 1,138원)",4.5,1641,1,NaN
